In [3]:
import multiprocessing 
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
documents2train = bdtesis.documents2train


In [70]:
#a minusculas
data = []
for x in bdtesis.documents2train.find():
    data.append(x['documento'])
datamin =[]
for x in (data):
    datamin.append(x.lower()) 


In [80]:
#limpieza de puntuación
import re
import nltk
import string
string.punctuation
datapunc = []
for x in datamin:
    datapunc.append(x.translate(str.maketrans('', '', string.punctuation)))


In [88]:
#retirar stopwords
from nltk.corpus import stopwords
stops = stopwords.words('spanish')
dataclean = []
for x in datapunc:
    palabras = x.split()
    resultwords  = [palabra for palabra in palabras if palabra.lower() not in stops]
    dataclean.append(' '.join(resultwords))
    


In [91]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataclean)]

In [92]:
#PARAMETROS
cores = multiprocessing.cpu_count()
words = 1
size = 300
context_window = 8
min_count = 19
epochs = 10
n_epochs = 10

In [93]:
n_epochs = 100
vec_size = 100
alpha = 0.025
model = Doc2Vec(size=vec_size, alpha=alpha,min_count=1, dm =0)
model.build_vocab(tagged_data)
%time model.train(tagged_data, total_examples=model.corpus_count, epochs=n_epochs)


model.save("./doc2vec_sbs_documents_pvdbow")
  

C:\Users\Carla\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Wall time: 19.9 s


In [4]:
model = Doc2Vec.load("./doc2vec_sbs_documents_pvdbow")


In [5]:
model.most_similar(positive=['seguros'],topn=10)

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cruzando', 0.3485018312931061),
 ('servirán', 0.31587108969688416),
 ('informados', 0.3111146092414856),
 ('fortuita', 0.30397242307662964),
 ('certifique', 0.3035879135131836),
 ('mínimas', 0.29421505331993103),
 ('combustibles', 0.2921939790248871),
 ('asalto', 0.28951817750930786),
 ('dieciocho', 0.28767311573028564),
 ('garantías', 0.28551140427589417)]